<h1> Calibration Comparison Bolaks - v1 </h1>

In [ ]:
import json, os
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
# from keras.models import load_model
import boto3
import pandas as pd
from PIL import Image
from collections import defaultdict


import numpy as np
from matplotlib import pyplot as plt
import time

pd.set_option('display.max_colwidth', 50)

<h1> Load and Clean Data </h1>

In [ ]:
def generate_comparison_results(key_a, key_b):
    csv_s3_url_a = output_csv_urls[key_a]
    s3_access_utils = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
    csv_f_a, bucket, key = s3_access_utils.download_from_url(csv_s3_url_a)
    df_original_a = pd.read_csv(csv_f_a)

    csv_s3_url_b = output_csv_urls[key_b]
    csv_f_b, bucket, key = s3_access_utils.download_from_url(csv_s3_url_b)
    df_original_b = pd.read_csv(csv_f_b)

    df_a = df_original_a.copy(deep=True)
    df_b = df_original_b.copy(deep=True)
    environment_a = [x for x in df_original_a.left_crop_url.iloc[0].split('/') if x.startswith('environment')][0]
    environment_b = [x for x in df_original_b.left_crop_url.iloc[0].split('/') if x.startswith('environment')][0]
    urls_a = list(df_a.left_crop_url.values)
    urls_b = list(df_b.left_crop_url.values)
    common_urls_a = list(set(urls_a).intersection(set([x.replace(environment_b, environment_a) for x in urls_b])))
    common_urls_b = list(set([x.replace(environment_a, environment_b) for x in urls_a]).intersection(set(urls_b)))
    df_a = df_a[df_a.left_crop_url.isin(common_urls_a)].sort_values(['captured_at', 'left_crop_url']).copy(deep=True)
    df_b = df_b[df_b.left_crop_url.isin(common_urls_b)].sort_values(['captured_at', 'left_crop_url']).copy(deep=True)
    df = pd.DataFrame({
        'captured_at': df_a.captured_at,
        'weight_a': df_a.estimated_weight_g.values,
        'weight_b': df_b.estimated_weight_g.values,
        'akpd_score_a': df_a.akpd_score.values,
        'akpd_score_b': df_b.akpd_score.values,
        'left_crop_url_a': df_a.left_crop_url.values,
        'right_crop_url_a': df_a.right_crop_url.values,
        'left_crop_url_post': df_b.left_crop_url.values,
        'right_crop_url_b': df_b.right_crop_url.values,
        'ann_a': df_a.annotation.values,
        'ann_b': df_b.annotation.values
    })

    df = df[(df.akpd_score_a > 0.9) & (df.akpd_score_b > 0.9)].copy(deep=True)
    
    print('{} parameters weight: {}g'.format(key_a, round(df_original_a[df_original_a.akpd_score > 0.9].estimated_weight_g.mean(), 2)))
    print('{} parameters weight: {}g'.format(key_b, round(df_original_b[df_original_b.akpd_score > 0.9].estimated_weight_g.mean(), 2)))
    print('--- APPLES TO APPLES COMPARISON')
    print('{} parameters weight: {}g'.format(key_a, round(df.weight_a.mean(), 2)))
    print('{} parameters weight: {}g'.format(key_b, round(df.weight_b.mean(), 2)))

    pct_difference = 100 * (df.weight_b.mean() - df.weight_a.mean()) / df.weight_a.mean()
    print('Percentage difference: {}%'.format(round(pct_difference, 2)))
    





In [ ]:
output_csv_urls = {
    'freshwater_matlab_pre_deployment': 'https://aquabyte-calibrations.s3-eu-west-1.amazonaws.com/biomass_experiments/bolaks.pen88.matlab.02042020.cal.output.csv',
    'freshwater_matlab_post_deployment': 'https://aquabyte-calibrations.s3-eu-west-1.amazonaws.com/biomass_experiments/bolaks.pen88.matlab.03112020.cal.output.csv',
    'freshwater_circular_post_deployment': 'https://aquabyte-calibrations.s3-eu-west-1.amazonaws.com/biomass_experiments/bolaks.pen88.circular.03112020.cal.output.csv',
    'cold_freshwater_matlab_post_deployment': 'https://aquabyte-calibrations.s3-eu-west-1.amazonaws.com/biomass_experiments/bolaks.pen88.matlab.03232020.cal.11degrees.output.csv'
}

In [ ]:
key_a, key_b = 'freshwater_matlab_pre_deployment', 'freshwater_matlab_post_deployment'
generate_comparison_results(key_a, key_b)


In [ ]:
key_a, key_b = 'freshwater_matlab_post_deployment', 'freshwater_circular_post_deployment'
generate_comparison_results(key_a, key_b)


In [ ]:
key_a, key_b = 'freshwater_matlab_post_deployment', 'cold_freshwater_matlab_post_deployment'
generate_comparison_results(key_a, key_b)


In [ ]:
key_a, key_b = 'freshwater_matlab_pre_deployment', 'cold_freshwater_matlab_post_deployment'
generate_comparison_results(key_a, key_b)


In [ ]:
csv_s3_url_a = output_csv_urls[key_a]
s3_access_utils = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
csv_f_a, bucket, key = s3_access_utils.download_from_url(csv_s3_url_a)
df_original_a = pd.read_csv(csv_f_a)

csv_s3_url_b = output_csv_urls[key_b]
csv_f_b, bucket, key = s3_access_utils.download_from_url(csv_s3_url_b)
df_original_b = pd.read_csv(csv_f_b)

df_a = df_original_a.copy(deep=True)
df_b = df_original_b.copy(deep=True)
environment_a = [x for x in df_original_a.left_crop_url.iloc[0].split('/') if x.startswith('environment')][0]
environment_b = [x for x in df_original_b.left_crop_url.iloc[0].split('/') if x.startswith('environment')][0]
urls_a = list(df_a.left_crop_url.values)
urls_b = list(df_b.left_crop_url.values)
common_urls_a = list(set(urls_a).intersection(set([x.replace(environment_b, environment_a) for x in urls_b])))
common_urls_b = list(set([x.replace(environment_a, environment_b) for x in urls_a]).intersection(set(urls_b)))
df_a = df_a[df_a.left_crop_url.isin(common_urls_a)].sort_values(['captured_at', 'left_crop_url']).copy(deep=True)
df_b = df_b[df_b.left_crop_url.isin(common_urls_b)].sort_values(['captured_at', 'left_crop_url']).copy(deep=True)
df = pd.DataFrame({
    'captured_at': df_a.captured_at,
    'weight_a': df_a.estimated_weight_g.values,
    'weight_b': df_b.estimated_weight_g.values,
    'akpd_score_a': df_a.akpd_score.values,
    'akpd_score_b': df_b.akpd_score.values,
    'left_crop_url_a': df_a.left_crop_url.values,
    'right_crop_url_a': df_a.right_crop_url.values,
    'left_crop_url_post': df_b.left_crop_url.values,
    'right_crop_url_b': df_b.right_crop_url.values,
    'ann_a': df_a.annotation.values,
    'ann_b': df_b.annotation.values
})

df = df[(df.akpd_score_a > 0.9) & (df.akpd_score_b > 0.9)].copy(deep=True)

print('{} parameters weight: {}g'.format(key_a, round(df.weight_a.mean(), 2)))
print('{} parameters weight: {}g'.format(key_b, round(df.weight_b.mean(), 2)))

pct_difference = 100 * (df.weight_b.mean() - df.weight_a.mean()) / df.weight_a.mean()
print('Percentage difference: {}%'.format(round(pct_difference, 2)))


In [ ]:
df_original_a.left_crop_url.iloc[0]

In [ ]:
df_original_b.left_crop_url.iloc[0]

In [ ]:
df.shape

<h1> Perform Un- and Re-Rectification </h1>

In [ ]:
from typing import Tuple
import numpy as np
import cv2

IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000

def get_camera_parameters(params: dict) -> Tuple:
    """Return individual camera parameters from JSON stereo parameters contents."""
    
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()
    
    imageSize = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, 
                                                               distCoeffs2, imageSize, R, T, None, None, 
                                                               None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1


def rectify_keypoints(annotation: dict, stereo_parameters: dict) -> dict:
    """Generated rectified key-points from unrectified key-point input and stereo parameters."""
    
    left_maps, right_maps, cameraMatrix1, distCoeffs1, R1, P1 = get_camera_parameters(stereo_parameters)
    
    rectified_ann = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in annotation[side]:
            x, y = item['xFrame'], item['yFrame']
            maps = left_maps_m if side == 'leftCrop' else right_maps_m
            x_new, y_new = cv2.undistortPoints(np.array([[[x, y]]]).astype(float), 
                                cameraMatrix1, distCoeffs1, R=R1, P=P1)[0][0]
            x_new, y_new = int(round(x_new)), int(round(y_new))
            rectified_ann[side].append({
                'keypointType': item['keypointType'],
                'xFrame': x_new,
                'yFrame': y_new
            })
            
    return rectified_ann



In [ ]:
matlab_params_f

In [ ]:
ann

In [ ]:
ann = json.loads(df.ann_a.iloc[0])
matlab_params = json.load(open(matlab_params_f))
rectify_keypoints(ann, matlab_params)

In [ ]:
ann

In [ ]:
s3_access_utils = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
import numpy as np
import cv2

In [ ]:
matlab_params_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40013180_R40029775/2020-01-14T00:00:00Z_L40013180_R40029775_stereo-parameters.json'
circular_params_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40013180_R40029775/2020-02-16T17:30:33.458096000Z_L40013180_R40029775_stereo-parameters.json'
matlab_params_f, _, _ = s3_access_utils.download_from_url(matlab_params_url)
circular_params_f, _, _ = s3_access_utils.download_from_url(circular_params_url)
left_maps_m, right_maps_m, cameraMatrix1_m, distCoeffs1_m, R1_m, P1_m = load_params(matlab_params_f)
left_maps_c, right_maps_c, cameraMatrix1_c, distCoeffs1_c, R1_c, P1_c = load_params(circular_params_f)

In [ ]:
ann_m_mp_c_cp_m_list = []
for idx, row in df.iterrows():
    ann_m = json.loads(row.ann_m)
    
    # un-rectify with matlab params, re-rectify with circular params
    ann_m_mp_c = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann_m[side]:
            bp = item['keypointType']
            x, y = item['xFrame'], item['yFrame']
            maps = left_maps_m if side == 'leftCrop' else right_maps_m
            x_new, y_new = cv2.undistortPoints(np.array([[maps[0][y, x]]]).astype(float), 
                                cameraMatrix1_c, distCoeffs1_c, R=R1_c, P=P1_c)[0][0]
            x_new, y_new = int(round(x_new)), int(round(y_new))
            ann_m_mp_c[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new
            })
            
    
    # now take above result, un-rectify with circular params, re-rectify with matlab params
    ann_m_mp_c_cp_m = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann_m_mp_c[side]:
            bp = item['keypointType']
            x, y = item['xFrame'], item['yFrame']
            maps = left_maps_c if side == 'leftCrop' else right_maps_c
            x_new, y_new = cv2.undistortPoints(np.array([[maps[0][y, x]]]).astype(float), 
                                cameraMatrix1_m, distCoeffs1_m, R=R1_m, P=P1_m)[0][0]
            x_new, y_new = int(round(x_new)), int(round(y_new))
            ann_m_mp_c_cp_m[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new
            })

    ann_m_mp_c_cp_m_list.append(ann_m_mp_c_cp_m)


In [ ]:
analysis_data = defaultdict(list)
for ann_m, ann_m_mp_c_cp_m in zip([json.loads(x) for x in list(df.ann_m.values)], ann_m_mp_c_cp_m_list):
    ann_m_left_kps = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_m['leftCrop']}
    ann_m_right_kps = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_m['rightCrop']}
    ann_m_mp_c_cp_m_left_kps = \
        {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_m_mp_c_cp_m['leftCrop']}
    ann_m_mp_c_cp_m_right_kps = \
        {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_m_mp_c_cp_m['rightCrop']}
    for bp in BODY_PARTS:
        analysis_data['body_part'].append(bp)
        analysis_data['x_1_l'].append(ann_m_left_kps[bp][0])
        analysis_data['y_1_l'].append(ann_m_left_kps[bp][1])
        analysis_data['x_1_r'].append(ann_m_right_kps[bp][0])
        analysis_data['y_1_r'].append(ann_m_right_kps[bp][1])
        analysis_data['x_2_l'].append(ann_m_mp_c_cp_m_left_kps[bp][0])
        analysis_data['y_2_l'].append(ann_m_mp_c_cp_m_left_kps[bp][1])
        analysis_data['x_2_r'].append(ann_m_mp_c_cp_m_right_kps[bp][0])
        analysis_data['y_2_r'].append(ann_m_mp_c_cp_m_right_kps[bp][1])
        
    

In [ ]:
analysis_df = pd.DataFrame(analysis_data)

In [ ]:
for bp in BODY_PARTS:
    body_part_mask = analysis_df.body_part == bp
    diffs = analysis_df[body_part_mask].x_1_l - analysis_df[body_part_mask].x_2_l
    print('Diffs in left crop x coordinate for {}: {}'.format(bp, diffs.abs().mean()))
    
    diffs = analysis_df[body_part_mask].y_1_l - analysis_df[body_part_mask].y_2_l
    print('Diffs in left crop y coordinate for {}: {}'.format(bp, diffs.abs().mean()))
    
    diffs = analysis_df[body_part_mask].x_1_r - analysis_df[body_part_mask].x_2_r
    print('Diffs in right crop x coordinate for {}: {}'.format(bp, diffs.abs().mean()))
    
    diffs = analysis_df[body_part_mask].y_1_r - analysis_df[body_part_mask].y_2_r
    print('Diffs in right crop y coordinate for {}: {}'.format(bp, diffs.abs().mean()))

In [ ]:
diffs = []
for idx, row in df.iterrows():
    ann_m = json.loads(row.ann_m)
    ann_dict_left_kps_m = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_m['leftCrop']}
    ann_dict_right_kps_m = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_m['rightCrop']}
    for bp in BODY_PARTS:
        diff = ann_dict_left_kps_m[bp][1] - ann_dict_right_kps_m[bp][1]
        diffs.append(diff)

print(np.median(diffs))

In [ ]:
diffs = []
for idx, row in df.iterrows():
    ann_c = json.loads(row.ann_c)
    ann_dict_left_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['leftCrop']}
    ann_dict_right_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['rightCrop']}
    for bp in BODY_PARTS:
        diff = ann_dict_left_kps_c[bp][1] - ann_dict_right_kps_c[bp][1]
        diffs.append(diff)
print(np.median(diffs))

In [ ]:
median_diffs = []
for idx, row in mdf.iterrows():
    ann_c = json.loads(row.annotation)
    ann_dict_left_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['leftCrop']}
    ann_dict_right_kps_c = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_c['rightCrop']}
    diffs = []
    for bp in BODY_PARTS:
        diff = ann_dict_left_kps_c[bp][1] - ann_dict_right_kps_c[bp][1]
        diffs.append(diff)
    median_diffs.append(np.median(diffs))
median_diffs = np.array(median_diffs)

In [ ]:
np.where(median_diffs > 15)

In [ ]:
mdf['median_diff'] = median_diffs

In [ ]:
kdf = mdf[(mdf.akpd_score > 0.95) & (mdf.median_diff > 15)].head(10)

<h1> Y-Coordinate Deviation Diagnosis </h1>

In [ ]:
tdf = pd.read_csv('/root/data/alok/biomass_estimation/playground/rectification.20200311.184020.MDSB.output.csv')
tdf = tdf.sort_values('captured_at').copy(deep=True)
tdf2 = pd.read_csv('/root/data/alok/biomass_estimation/playground/rectification_2.csv')
tdf2 = tdf2.sort_values('captured_at').copy(deep=True)
tdf['annotation'] = tdf2.annotation.values


In [ ]:
json.loads(tdf2[tdf2.captured_at == '2019-09-13T01:15:56.350510000Z'].annotation.iloc[0])

In [ ]:
json.loads(tdf[tdf.captured_at == '2019-09-13T01:15:56.350510000Z'].left_crop_metadata.iloc[0])

In [ ]:
tdf2.annotation.iloc[1]

In [ ]:
tdf.annotation.iloc[1]

In [ ]:
def load_params(params):
    print("Loading params...")
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)

    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)

    print("Params loaded.")
    return left_maps, right_maps

def rectify_crop(crop, maps, crop_metadata):
    print("Rectifying...")
    new_image = np.zeros([3000, 4096, 3]).astype('uint8')
    lower_left = (crop_metadata['y_coord'] + crop_metadata['height'], crop_metadata['x_coord'])
    upper_right = (crop_metadata['y_coord'], crop_metadata['x_coord'] + crop_metadata['width'])
    new_image[upper_right[0]:lower_left[0], lower_left[1]:upper_right[1], :] = np.array(crop)
    remap = cv2.remap(new_image, maps[0], maps[1], cv2.INTER_LANCZOS4)
    nonzero_indices = np.where(remap > 0)
    y_min, y_max = nonzero_indices[0].min(), nonzero_indices[0].max()
    x_min, x_max = nonzero_indices[1].min(), nonzero_indices[1].max()
    lower_left = (y_max, x_min)
    upper_right = (y_min, x_max)
    rectified_crop = remap[upper_right[0]:lower_left[0], lower_left[1]:upper_right[1], :].copy()

    # construct rectified crop metadata
    rectified_crop_metadata = crop_metadata.copy()
    rectified_crop_metadata['x_coord'] = int(x_min)
    rectified_crop_metadata['y_coord'] = int(y_min)
    rectified_crop_metadata['width'] = int(x_max - x_min)
    rectified_crop_metadata['height'] = int(y_max - y_min)

    print("Rectification done")
    return rectified_crop, rectified_crop_metadata


def create_crop_metadata(raw_crop_f):
    coords = [int(x) for x in os.path.basename(raw_crop_f).replace('.jpg', '').split('_')[-4:]]
    
    crop_metadata = {}
    crop_metadata['x_coord'] = coords[0]
    crop_metadata['y_coord'] = coords[1]
    crop_metadata['width'] = coords[2] - coords[0]
    crop_metadata['height'] = coords[3] - coords[1]
    print(coords[0])
    
    return crop_metadata

def display_crops(left_image, right_image, ann, overlay_keypoints=True, show_labels=False):
    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    
    left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['leftCrop']}
    right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['rightCrop']}
    if overlay_keypoints:
        for bp, kp in left_keypoints.items():
            axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in right_keypoints.items():
            axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
# load maps
matlab_stereo_parameters_url = 'https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40013180_R40029775/2020-01-14T00:00:00Z_L40013180_R40029775_stereo-parameters.json'
matlab_stereo_params_f, _, _ = s3_access_utils.download_from_url(matlab_stereo_parameters_url)
matlab_stereo_params = json.load(open(matlab_stereo_params_f))
left_maps, right_maps = load_params(matlab_stereo_params)

# load crops and metadata
i = 2
left_crop_url = tdf.left_crop_url.iloc[i]
right_crop_url = tdf.right_crop_url.iloc[i]
ann = json.loads(tdf.annotation.iloc[i])
crops_json_url_base = os.path.dirname(left_crop_url.replace('aquabyte-crops-test', 'aquabyte-frames-resized-inbound'))
crops_json_url = os.path.join(crops_json_url_base, 'crops.json')
crops_json_f, _, _ = s3_access_utils.download_from_url(crops_json_url)
left_raw_crop_url = left_crop_url.replace('aquabyte-crops-test', 'aquabyte-frames-resized-inbound')
left_raw_crop_f, _, _ = s3_access_utils.download_from_url(left_raw_crop_url)
left_raw_crop = Image.open(left_raw_crop_f)
right_raw_crop_url = right_crop_url.replace('aquabyte-crops-test', 'aquabyte-frames-resized-inbound')
right_raw_crop_f, _, _ = s3_access_utils.download_from_url(right_raw_crop_url)
right_raw_crop = Image.open(right_raw_crop_f)

left_crop_metadata = create_crop_metadata(left_raw_crop_f)
right_crop_metadata = create_crop_metadata(right_raw_crop_f)

rectified_left_crop, rectified_left_crop_metadata = rectify_crop(left_raw_crop, left_maps, left_crop_metadata)
rectified_right_crop, rectified_right_crop_metadata = rectify_crop(right_raw_crop, right_maps, right_crop_metadata)

In [ ]:
display_crops(rectified_left_crop, rectified_right_crop, ann)

In [ ]:
left_crop_url

In [ ]:
left_raw_crop_url

In [ ]:
odf = pd.read_csv('/root/data/alok/biomass_estimation/playground/pen=61/data_dump_1.csv')

In [ ]:
odf.captured_at

In [ ]:
df.index = pd.to_datetime(df.captured_at)
odf.index = pd.to_datetime(odf.captured_at)

In [ ]:
odf['left_crop_fname'] = odf.left_crop_url.apply(lambda x: os.path.basename(x))
df['left_crop_fname'] = df.left_crop_url_m.apply(lambda x: os.path.basename(x))


In [ ]:
df.index = df.left_crop_fname
odf.index = odf.left_crop_fname

In [ ]:
left_diffs, right_diffs = [], []
count = 0
for idx, row in df.iterrows():
    if count % 100 == 0:
        print(count)
    count += 1
    ann_1 = json.loads(row.ann_m)
    if (odf.left_crop_fname == row.left_crop_fname).sum() > 0:
        ann_2 = json.loads(odf[odf.left_crop_fname == row.left_crop_fname].annotation.iloc[0])
        ann_1_left_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_1['leftCrop']}
        ann_1_right_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_1['rightCrop']}
        ann_2_left_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_2['leftCrop']}
        ann_2_right_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann_2['rightCrop']}
        for bp in BODY_PARTS:
            left_diffs.append(ann_1_left_dict[bp][1] - ann_2_left_dict[bp][1])
            right_diffs.append(ann_1_right_dict[bp][1] - ann_2_right_dict[bp][1])


In [ ]:
np.mean(right_diffs)